In [1]:
from geminiFunctions import *
from prompts_and_sys_instructions import *
import json

with open("attractions.txt", encoding="utf-8") as f:
    attractions = f.read()

with open("init_chat_history.txt") as f:
    chat_history = (f.read())

def retry_until_success(func, *args):
    while True:
        try:
            return func(*args)
        except Exception as e:
            print(f"Error: {e}. Retrying...")
def shortlist_attractions(attractions_list, chat_history):
    system_instruction = system_instruction_for_shortlisting_attractions
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given chat history and attractions list, shortlist the attractions. Attractions list : {attractions_list} \n\n chat history : {chat_history} ", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]


def cluster_groups_by_geographical_data(attractions_list, chat_history):
    system_instruction = system_instruction_for_clustering_attractions_geographically
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given chat history and attractions list, cluster the attractions based on their geographical data. Attractions list : {attractions_list} \n\n chat history : {chat_history} ", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]

def get_timings_for_attractions(attractions_list, chat_history):
    system_instruction = system_instruction_for_getting_best_times_to_visit
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given chat history and attractions list, give the best times for the attractions based on the api results and google saerch and your training data. Attractions list : {attractions_list} \n\n chat history : {chat_history} ", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]

def get_budget_reasoning_for_attractions(attractions_list, chat_history):
    system_instruction = system_instruction_for_budget_reasoning
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given chat history and attractions list, give the budget based attractions. Attractions list : {attractions_list} \n\n chat history : {chat_history} ", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]

def get_day_wise_itinerary(chat_history, geographical_reasoning, time_based_reasoning, budget_reasoning, attractions, shortlisted_attractions, duration_analysis):
    system_instruction = system_instruction_for_day_wise_planning
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given chat history, geographical reasoning, time based reasoning ,budget reasoning, duration analysis, original(complete attractions list) and shortlisted_attraction list(done by another llm based on what it thought are user's preferences) give the day wise itinerary. \n\n chat history : {chat_history} \n\n geographical reasoning : {geographical_reasoning} \n\n time based reasoning : {time_based_reasoning} \n\n budget reasoning : {budget_reasoning} \n\n duration analysis : {duration_analysis} \n\n original attractions list : {attractions}, \n\n shortlisted attractions : {shortlisted_attractions}", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]

def summarise_attractions(attractions_list, chat_history):
    system_instruction = system_instruction_for_summarising_search_json
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given chat history and attractions list, summarise the attractions. Keep in mind that your output is the input for other llm agents which will perform further analysis on the attractions so your description of the attraction must be detailed enough for it to be helpful to any and all cases of analysis. Attractions list : {attractions_list} \n\n chat history : {chat_history} ", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]

def remove_redundant_attractions(summarised_attractions_list, chat_history):
    system_instruction = system_instruction_for_removing_redundant_attractions
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given chat history and attractions list, remove the redundant attractions. Attractions list : {summarised_attractions_list} \n\n chat history : {chat_history} ", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]

def get_duration_analysis_of_attractions(summarised_attractions, chat_history):
    system_instruction = system_instruction_for_duration_analysis
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given chat history and attractions list, give the duration analysis of the attractions. Attractions list : {summarised_attractions} \n\n chat history : {chat_history} ", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]

def get_intraday_planning(attractions, chat_history, day_wise_itinerary, duration_analysis, clustered_attractions, time_based_attrations):
    system_instruction = system_instruction_for_intraday_planning
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given chat history, attractions, day wise itinerary, duration analysis, clustered attractions, time based attractions and budget reasoned attractions, give the intraday planning. \n\n chat history : {chat_history} \n\n attractions : {attractions} \n\n day wise itinerary : {day_wise_itinerary} \n\n duration analysis : {duration_analysis} \n\n clustered attractions : {clustered_attractions} \n\n time based attractions : {time_based_attrations} \n\n", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]

def add_free_attractions_on_route(chat_history, day_wise_itinerary):
    system_instruction = system_instruction_for_free_attraction_addition
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given chat history and day wise itinerary, add the free attractions on the route. \n\n chat history : {chat_history} \n\n day wise itinerary : {day_wise_itinerary} ", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]

def get_route_to_be_followed(chat_history, current_itinerary):
    system_instruction = system_instruction_for_route_planning
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given chat history and current itinerary, give the route to be followed. \n\n chat history : {chat_history} \n\n current itinerary : {current_itinerary} ", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]

def add_restaurants_on_route(chat_history, current_itinerary):
    system_instruction = system_instruction_for_adding_restaurants_on_the_route
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given chat history and current itinerary, add the restaurants on the route. \n\n chat history : {chat_history} \n\n current itinerary : {current_itinerary} ", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]

def get_itinerary_json(itinerary, attractions, chat_history):
    system_instruction = system_instruction_for_getting_itinerary_json
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given chat history, itinerary, and tbo's api data for searching attractions in the given city, create the json. \n\n tbo's api result : {attractions} \n\n itinerary : {itinerary}, \n\n chat history : {chat_history} ", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]

def get_tbo_description(output_json, attractions):
    system_instruction = system_instruction_for_adding_tbo_description
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given output json and attractions, add the tbo description to the attractions. \n\n output json : {output_json} \n\n attractions : {attractions} ", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]

def get_llm_description(chat_history, itinerary):
    system_instruction = system_instruction_for_getting_llm_justifications
    history, chat, _ = start_chat(system_instruction)
    history, chat = send_message(f"Based on the given chat history and itinerary, give the llm description. \n\n chat history : {chat_history} \n\n itinerary : {itinerary} ", history, chat, system_instruction)
    return history[-1]["parts"][0]["text"]

import json

def populate_tbo_descriptions(itinerary_json, tbo_description_json):
    """
    Populates the 'tbo_description' field of each attraction in the itinerary with a summary from the corresponding entry in the tbo description JSON.

    Args:
        itinerary_json (str): A JSON string representing the complete itinerary where the key is complete_itinerary.
        tbo_description_json (str): A JSON string where keys are attraction names and values are the tbo descriptions.

    Returns:
        str: A JSON string representing the updated itinerary JSON with populated 'tbo_description' fields.
    """

    try:
        itinerary = json.loads(itinerary_json)
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding itinerary JSON: {e}")

    try:
        tbo_descriptions = json.loads(tbo_description_json)
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding TBO descriptions JSON: {e}")

    if 'complete_itinerary' not in itinerary:
        raise ValueError("Invalid itinerary json, 'complete_itinerary' key not found")

    for day, day_data in itinerary['complete_itinerary'].items():
        if isinstance(day_data, list):
            for activity in day_data:
                if 'SightseeingName' in activity and activity['SightseeingName'] in tbo_descriptions:
                    activity['tbo_description'] = tbo_descriptions[activity['SightseeingName']]
                else:
                    print(f"Warning: No matching TBO description found for activity: {activity.get('SightseeingName', 'Unknown')}, skipped")
        elif isinstance(day_data, dict) and 'intraday_plan' in day_data :
            for time_slot, slot_data in day_data['intraday_plan'].items():
                if isinstance(slot_data, dict) and 'attractions' in slot_data:
                    for activity in slot_data['attractions']:
                        if 'SightseeingName' in activity and activity['SightseeingName'] in tbo_descriptions:
                            activity['tbo_description'] = tbo_descriptions[activity['SightseeingName']]
                        else:
                            print(f"Warning: No matching TBO description found for activity: {activity.get('SightseeingName', 'Unknown')}, skipped")

    return json.dumps(itinerary, indent=2)
def populate_llm_descriptions(itinerary_json, llm_description_json):
    """
    Populates the 'llm_description' field of each attraction in a TBO-like itinerary JSON object.

    Args:
        itinerary_json (str): A JSON string representing a complete itinerary, with TBO-like structure.
        llm_description_json (str): A JSON string where keys are attraction names and values are personalized justifications.

    Returns:
        str: A JSON string representing the updated itinerary JSON with populated 'llm_description' fields.
    """

    try:
        itinerary = json.loads(itinerary_json)
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding itinerary JSON: {e}")

    try:
        llm_descriptions = json.loads(llm_description_json)
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding LLM descriptions JSON: {e}")

    if 'complete_itinerary' not in itinerary:
        raise ValueError("Invalid itinerary json, 'complete_itinerary' key not found")


    for day, day_data in itinerary['complete_itinerary'].items():
        if isinstance(day_data, list):
            for activity in day_data:
                if 'SightseeingName' in activity and activity['SightseeingName'] in llm_descriptions:
                    activity['llm_description'] = llm_descriptions[activity['SightseeingName']]
                else:
                    print(f"Warning: No matching LLM description found for activity: {activity.get('SightseeingName', 'Unknown')}, skipping.")
        elif isinstance(day_data, dict) and 'intraday_plan' in day_data:
            for time_slot, slot_data in day_data['intraday_plan'].items():
                if isinstance(slot_data, dict) and 'attractions' in slot_data:
                    for activity in slot_data['attractions']:
                        if 'SightseeingName' in activity and activity['SightseeingName'] in llm_descriptions:
                            activity['llm_description'] = llm_descriptions[activity['SightseeingName']]
                        else:
                            print(f"Warning: No matching LLM description found for activity: {activity.get('SightseeingName', 'Unknown')}, skipping.")
    return json.dumps(itinerary, indent=2)




print(attractions)


[{'Response': {'ResponseStatus': 1, 'Error': {'ErrorCode': 0, 'ErrorMessage': ''}, 'TraceId': '75e8c9c5-3b78-47f3-a7c2-44f225e32869', 'PreferredCurrency': 'INR', 'SightseeingSearchResults': [{'ResultIndex': 68, 'CityId': '115936', 'CityName': 'Dubai', 'CountryCode': 'AE', 'FromDate': '2025-03-31T00:00:00', 'ToDate': '2025-03-31T00:00:00', 'SightseeingName': 'Classical Dubai City - Half-Day Tour', 'SightseeingCode': 'E-E10-A1MANO0376', 'SightseeingTypes': ['TOURS'], 'DurationDescription': [{'TotalDuration': '1 DAYS', 'Date': '2025-03-31T00:00:00'}], 'Condition': 'Printed Voucher. Print and bring the voucher to enjoy the activity. ', 'AdditionalInformation': 'Please make sure you answer all the mandatory questions during the booking process.', 'ImageList': ['https://media.activitiesbank.com/62291/ENG/B/1.jpg', 'https://media.activitiesbank.com/62291/ENG/B/24.jpg', 'https://media.activitiesbank.com/62291/ENG/B/18.jpg', 'https://media.activitiesbank.com/62291/ENG/B/12.jpg', 'https://media.

In [2]:
summarised_attractions = retry_until_success(summarise_attractions, attractions, chat_history)


Gemini Response: ```json
{
   "attraction_summary": "[{\"attraction_summary\": \"SightseeingName: Classical Dubai City - Half-Day Tour, SightseeingCode: E-E10-A1MANO0376, City: Dubai, Country: AE; Price: INR 1082.37 (Offered), Base: 1443.16, Currency: INR; Duration: 1 day; Incl: Guide, Hotel pick-up/drop-off. Key Details: Half-day tour starting at Zabeel Palace, with stops at Dubai Frame, Bastakiya, Abra station, Spice & Gold Souqs, Islamic Art Centre, Burj Al Arab & The Pointe. Excl: Entrance to Frame, Lunch/drinks; Condition: Printed voucher required. Expect a comprehensive tour of Dubai's key landmarks and cultural sites, with photo stops, market visits, and a traditional boat ride, and the tour will take approximately half a day. This tour is best suited for first time visitors who want to get an overview of the city.\", \"images\": [\"https://media.activitiesbank.com/62291/ENG/B/1.jpg\"]},{\"attraction_summary\": \"SightseeingName: Traditional Dubai - Half-Day City Tour, Sightseei

In [3]:
redundancy_removed_attractions = retry_until_success(remove_redundant_attractions, summarised_attractions, chat_history)




Gemini Response: ```json
{
  "filtered_attractions": [
    {
      "SightseeingName": "Traditional Dubai - Half-Day City Tour",
      "reason": "Selected this half-day tour because it focuses on the historical aspects of Dubai which is a 'must do' and is suitable for older travellers with a slow pace, and is a great way to get an overview of Old Dubai. The other half day tour was removed due to it covering very similar attractions (like Dubai Frame), and also because this tour has better user reviews on Google. The other tour also had a visit to Zabeel Palace, which is not as iconic as the Bastakiya area, and is not considered a 'must-do' for older travellers."
    },
    {
      "SightseeingName": "Dhow Cruise with dinner - Dubai Creek",
      "reason": "Selected this dinner cruise as it offers a relaxed evening with views of Dubai Creek, dinner, and live entertainment which is suitable for older travelers. The other options for dinner cruises on Dubai Marina were removed because this

In [4]:
shortlisted_attractions = retry_until_success(shortlist_attractions, redundancy_removed_attractions, chat_history)



Gemini Response: ```json
{
  "shortlisted_attractions": [
    {
      "SightseeingName": "Traditional Dubai - Half-Day City Tour",
      "reason": "Ranked 1st as it matches the user's explicit preference for a slow-paced, relaxing tour with cultural experiences, and it is a 'must do' tour of the city. Google user reviews also suggest this is a good option for older travelers. The tour focuses on the historical aspects of Dubai, which is more aligned with the user's interests."
    },
     {
      "SightseeingName": "At the Top Burj Khalifa: Levels 124 and 125 with Transfer",
      "reason": "Ranked 2nd as it is an iconic landmark and a 'must do' activity in Dubai that matches user preference, and is suitable for older travelers as the access is mostly through elevators. Google user reviews suggest it is a must do for all visitors."
    },
     {
      "SightseeingName": "Miracle Garden Visit with Transfers - Ticket",
      "reason": "Ranked 3rd as it is a beautiful and serene attractio

In [5]:
duration_analysis = retry_until_success(get_duration_analysis_of_attractions, shortlisted_attractions, chat_history)
clustered_attractions = retry_until_success(cluster_groups_by_geographical_data, shortlisted_attractions, chat_history)
time_based_attractions = retry_until_success(get_timings_for_attractions, shortlisted_attractions, chat_history)
budget_reasoned_attractions = retry_until_success(get_budget_reasoning_for_attractions, shortlisted_attractions, chat_history)

day_wise_itinerary = retry_until_success(get_day_wise_itinerary, chat_history, clustered_attractions, time_based_attractions, budget_reasoned_attractions, attractions, shortlisted_attractions, duration_analysis)


Gemini Response: ```json
{
  "recommended_durations": {
    "Traditional Dubai - Half-Day City Tour": {
      "recommended_time_range": "4-5 hours",
      "reason": "The user prefers a slow pace, and the tour is a half-day tour.  Based on google searches, typical half day city tours last around 3-4 hours. Given the user's preference for a relaxed pace, and the fact that they are older, I've added extra time for breaks and to explore each location at a comfortable pace. Also, since the user prefers a slow pace, they may take more time at each attraction."
    },
    "At the Top Burj Khalifa: Levels 124 and 125 with Transfer": {
      "recommended_time_range": "2-3 hours",
      "reason": "User reviews on Google suggest that a visit to the Burj Khalifa typically takes 1.5 - 2 hours. Since the user prefers a slow pace and is older, I have added extra buffer time. The transfer time has also been added to this time range. This time is sufficient for enjoying the views, taking photographs, a

In [6]:
free_attractions_added = retry_until_success(add_free_attractions_on_route, chat_history, day_wise_itinerary)


Gemini Response: ```json
{
  "augmented_itinerary": {
    "day1": [
      {
        "SightseeingName": "Al Seef District",
        "reason": "Added because it is a free heritage area near the Spice Souk and Gold Souk (which are part of the existing itinerary for this day), and it also fits the user’s preference for a cultural experience and shopping. Google search shows it has a mix of traditional and modern elements with shops and cafes, and the timings are flexible so it fits well with the day’s existing plan. User reviews are also good for this location and the location is very close to the gold and spice souks."
      }
    ],
    "day2": [
      {
        "SightseeingName": "Jumeirah Public Beach",
        "reason": "Added as this free beach is near the Miracle Garden (which is already part of the day’s plan), and it offers a relaxing environment. Google search shows the beach is open all the time, and the user can enjoy some time there after their garden visit. It also has good u

In [7]:
intraday_planning = retry_until_success(get_intraday_planning, attractions, chat_history, day_wise_itinerary, duration_analysis, clustered_attractions, time_based_attractions)
route_to_be_followed = retry_until_success(get_route_to_be_followed, chat_history, intraday_planning)
restaurants_added = retry_until_success(add_restaurants_on_route, chat_history, route_to_be_followed)



Gemini Response: ```json
{
"multi_day_itinerary":{
  "day1": {
      "intraday_plan": {
          "8:50 PM - 9:30 PM": {
              "action": "Arrive at Dubai Airport and transfer to Hotel",
              "details": "Arrive at Dubai International Airport at 8:50 PM. Proceed through immigration and baggage claim, and take a pre-booked taxi or hotel transfer to Radisson Blu Hotel. This transfer is not included in the activities, and must be arranged separately. A buffer of 40 minutes is given, assuming the transfer will take around 30 minutes. Google search shows that there is no significant traffic at this time."
          },
          "9:30 PM - 10:30 PM": {
              "action": "Check into Hotel and Relax",
              "details": "Check into Radisson Blu Hotel and take some time to settle in and relax after your journey. This allows for a smooth and comfortable start to the trip. A buffer of 1 hour is given."
           }
        }
    },
   "day2":{
        "intraday_plan":{


In [8]:
print(system_instruction_for_getting_itinerary_json)

You are a highly meticulous and precise itinerary formatter for TBO.com. Your sole task is to take the processed outputs from various LLM nodes and generate a final, structured JSON object suitable for display on a frontend, while also handling cases where an attraction is not present in the TBO response list. This output should include all essential details about attractions, and should include the following fields: tbo_description (which must have an empty string as its value), llm_description (which must have an empty string as its value), and llm_description_one_liner (which must have an empty string as its value). You will be provided with:

A JSON object representing a multi-day intraday itinerary with restaurants: This is the output from the restaurant recommendation LLM, which contains the detailed routes for each day, the attractions with time ranges, and any restaurant recommendations.

A JSON object containing day-wise plans of attractions: This is the output of the master i

In [13]:
print(chat_history)


[{'role': 'user', 'parts': [{'text': 'hi there'}]}, {'role': 'model', 'parts': [{'text': "Hello there! Welcome to TBO.com, I'm super excited to be your travel buddy today! Let's dive into the fun stuff – tell me, where in the world are you dreaming of going? What's the destination that's been on your mind lately?\n"}]}, {'role': 'user', 'parts': [{'text': "I'm planning a trip to Dubai"}]}, {'role': 'model', 'parts': [{'text': "Dubai! Oh, that's a fantastic choice! Get ready for dazzling skyscrapers, luxurious experiences, and a whole lot of excitement! To start, tell me a bit more about your trip. What kind of adventure are you hoping for in Dubai? Are you picturing a relaxing getaway, an action-packed exploration, or perhaps a mix of both?\n"}]}, {'role': 'user', 'parts': [{'text': 'We are old people so yes, we need something relaxing and slow paced'}]}, {'role': 'model', 'parts': [{'text': 'Okay, a relaxing and slow-paced trip to Dubai sounds absolutely wonderful! Picture this: gentl

In [9]:
output_json = retry_until_success(get_itinerary_json, restaurants_added, attractions, chat_history)



Gemini Response: ```json
{
  "complete_itinerary": {
    "day1": [
      {
        "SightseeingName": "Arrive at Dubai Airport and transfer to Hotel",
        "SightseeingCode": null,
        "time_range": "8:50 PM - 9:30 PM",
        "price": 0,
        "currency": null,
        "TourDescription": "",
         "next_action": "travel",
         "ends_next_day": false,
          "restaurant_reason": "",
           "llm_description": "",
            "llm_description_one_liner":""
      },
       {
        "SightseeingName": "Check into Hotel and Relax",
        "SightseeingCode": null,
        "time_range": "9:30 PM - 10:30 PM",
         "price": 0,
          "currency": null,
         "TourDescription": "",
         "next_action": "rest",
          "ends_next_day": false,
          "restaurant_reason": "",
            "llm_description": "",
             "llm_description_one_liner":""
      }
    ],
    "day2": [
      {
         "SightseeingName": "Breakfast at the hotel",
        "Sigh

In [10]:
str_json = str(output_json)
if(str_json[0] == '`'):
    str_json = str_json[7:]
    str_json = str_json[:-4]
    output_json = convert_string_to_json(str_json)



SyntaxError: '{' was never closed (<string>, line 713)

In [ ]:
tbo_descriptions = get_tbo_description(output_json, attractions)


In [ ]:
output_json = populate_tbo_descriptions(output_json, tbo_descriptions)



In [ ]:
llm_descriptions = get_llm_description(chat_history, output_json)


In [ ]:
output_json = populate_llm_descriptions(output_json, llm_descriptions)


In [ ]:
with open("itinerary.json", "w") as f:
    json.dump(output_json, f)